In [ ]:
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import pandas as pd
from datetime import datetime
import os
import importlib
import qlbmlib
from skimage.measure import marching_cubes
importlib.reload(qlbmlib)

<module 'qlbmlib' from 'd:\\Data\\Codes\\Quantum\\Research\\qlbmlib.py'>

In [3]:
def get_3d_planes_initial_distribution(sites: tuple[int, int, int], 
                                     plane_width: int = 2,
                                     background_density: float = 0.1,
                                     plane_density: float = 0.9) -> NDArray[np.float64]:
    """
    Create a distribution with three intersecting planes at the center.
    
    Args:
        sites: Tuple of (depth, height, width) for the 3D grid
        plane_width: Width of the planes in lattice units
        background_density: Density value outside the planes
        plane_density: Density value in the planes
        
    Returns:
        density: Initial density distribution with three intersecting planes
    """
    # Initialize density array with background value
    density = np.full(sites, background_density)
    
    # Calculate center indices (subtract 1 to match provided initialization)
    center_x = sites[0] // 2 - 1
    center_y = sites[1] // 2 - 1
    center_z = sites[2] // 2 - 1
    
    # Create the three orthogonal planes by setting specific indices
    # YZ plane (fixed x)
    density[center_x, :, :] = plane_density
    
    # XZ plane (fixed y)
    density[:, center_y, :] = plane_density
    
    # XY plane (fixed z)
    density[:, :, center_z] = plane_density
    
    return density

def get_3d_velocity_field(sites: tuple[int, int, int], 
                        A: float = 0.2, B: float = 0.2, C: float = 0.2,
                        a: float = 1.0, b: float = 1.0, c: float = 1.0) -> NDArray[np.float64]:
    """
    Create a 3D velocity field based on trigonometric functions with separate amplitude and frequency parameters.
    
    Args:
        sites: Tuple of (depth, height, width) for the 3D grid
        A, B, C: Amplitude parameters for each velocity component (default: 0.2)
        a, b, c: Frequency parameters for each dimension (default: 1.0)
        
    Returns:
        velocity_field: Array of shape (3, depth, height, width) containing velocity components
    """
    # Create coordinate grids from 0 to 2π
    x = np.linspace(0, 2*np.pi, sites[0])
    y = np.linspace(0, 2*np.pi, sites[1])
    z = np.linspace(0, 2*np.pi, sites[2])
    
    # Initialize velocity components
    u = np.zeros(sites)
    v = np.zeros(sites)
    w = np.zeros(sites)
    
    # Populate the velocity field
    for i in range(sites[0]):
        for j in range(sites[1]):
            for k in range(sites[2]):
                u[i,j,k] = A * np.cos(a * x[i]) * np.sin(b * y[j]) * np.sin(c * z[k])
                v[i,j,k] = B * np.sin(a * x[i]) * np.cos(b * y[j]) * np.sin(c * z[k])
                w[i,j,k] = C * np.sin(a * x[i]) * np.sin(b * y[j]) * np.cos(c * z[k])
    
    return np.stack([u, v, w])



In [4]:
# Create 3D configuration
sites_3d = (8, 8, 8)

# D3Q27 lattice configuration:
# - Each velocity vector is a combination of [-1,0,1] in each dimension
# - Total 27 vectors: 1 rest (0,0,0), 6 face neighbors, 12 edge neighbors, 8 corner neighbors
links = [
    [0,0,0], # Rest particle
    [-1,0,0], [1,0,0], [0,-1,0], [0,1,0], [0,0,-1], [0,0,1], 
    [-1,-1,0], [-1,1,0], [1,-1,0], [1,1,0], 
    [-1,0,-1], [-1,0,1], [1,0,-1], [1,0,1], 
    [0,-1,-1], [0,-1,1], [0,1,-1], [0,1,1],
    [-1,-1,-1], [-1,-1,1], [-1,1,-1], [-1,1,1],
    [1,-1,-1], [1,-1,1], [1,1,-1], [1,1,1]
]

# Weights based on distance from center:
# - Rest particle (0,0,0): 1/27
# - Face neighbors: 1/54 (6 directions)
# - Edge neighbors: 1/108 (12 directions)
# - Corner neighbors: 1/216 (8 directions)
weights = [
    8/27 if all(v == 0 for v in link) else  # center
    2/27 if sum(abs(v) for v in link) == 1 else  # face neighbors
    1/54 if sum(abs(v) for v in link) == 2 else  # edge neighbors
    1/216  # corner neighbors
    for link in links
]

speed_of_sound = 1/np.sqrt(3)

# Create initial distribution with three intersecting planes
initial_dist_3d = get_3d_planes_initial_distribution(sites_3d, plane_width=2)

# Create velocity field
velocity_field = get_3d_velocity_field(sites_3d, A=0.2, B=0.2, C=0.2, a=1.0, b=1.0, c=1.0)

# Configuration for simulation (30 iterations)
config_3d = [(10, velocity_field, links, weights, speed_of_sound)]

# Create experiment directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
dirname = f"experiments/3DQ27Planes_{timestamp}"
os.makedirs(dirname, exist_ok=True)

# Define output files
classical_csv = f"{dirname}/classical.csv"
quantum_csv = f"{dirname}/quantum.csv"
rmse_plot = f"{dirname}/rmse_plot.png"



In [18]:
def save_isosurface_plots(data_file: str, sites: tuple[int, int, int], output_fig: str, 
                        timesteps: list[int], n_surfaces: int = 3, 
                        figsize: tuple[int, int] = (15, 12),
                        view_angles: tuple[float, float] = (45, 45)):
    """
    Create and save a 2x2 grid of isosurface plots for specified timesteps.
    
    Args:
        data_file: Path to CSV file containing flattened density data
        sites: Tuple of (depth, height, width) for the 3D grid
        timesteps: List of 4 timesteps to plot
        n_surfaces: Number of isosurfaces to plot (default: 3)
        figsize: Figure size in inches (default: (15, 12))
        view_angles: Tuple of (elevation, azimuth) angles for 3D view (default: (45, 45))
    """
    
    # Read the data without headers
    df = pd.read_csv(data_file, header=None)
    if max(timesteps) >= len(df):
        raise ValueError(f"Max timestep {max(timesteps)} exceeds data length {len(df)}")
    
    # Create figure with proper spacing
    fig = plt.figure(figsize=figsize)
    gs = plt.GridSpec(2, 3, figure=fig, width_ratios=[1, 1, 0.15])
    
    # Get data range for each timestep to pick meaningful isovalues
    timestep_data = [df.iloc[t].values for t in timesteps]
    data_mins = [data.min() for data in timestep_data]
    data_maxs = [data.max() for data in timestep_data]
    global_min = min(data_mins)
    global_max = max(data_maxs)
    
    # Create logarithmically spaced isovalues for better density representation
    isovalues = np.logspace(np.log10(global_min), np.log10(global_max), n_surfaces + 2)[1:-1]
    
    # Create colormap and calculate transparencies
    cmap = plt.cm.viridis
    normalized_positions = np.linspace(0, 1, n_surfaces)
    colors = [cmap(i) for i in normalized_positions]
    
    # Calculate alphas proportional to density
    # Higher density regions are more opaque
    alphas = np.linspace(0.2, 0.8, n_surfaces)  # from opaque (0.2) to transparent (0.8)
    
    # Plot each timestep
    for idx, timestep in enumerate(timesteps):
        # Print debug info
        print(f"Time step {timestep}:")
        print(f"  Density range: [{data_mins[idx]:.3f}, {data_maxs[idx]:.3f}]")
        print(f"  Using isosurface levels: {[f'{v:.3f}' for v in isovalues]}")
        
        # Calculate subplot position
        row = idx // 2
        col = idx % 2
        
        # Create subplot
        ax = fig.add_subplot(gs[row, col], projection='3d')
        
        # Get density data for this timestep and reshape to 3D grid
        density_data = df.iloc[timestep].values
        try:
            density_grid = density_data.reshape(sites, order='F')  # Use Fortran order to match data
        except ValueError as e:
            raise ValueError(f"Could not reshape data of length {len(density_data)} into grid of shape {sites}") from e
        
        # Plot multiple isosurfaces
        for isovalue, color, alpha in zip(isovalues, colors, alphas):
            try:
                # Use spacing parameter to handle non-cubic grids
                spacing = (1.0, 1.0, 1.0)  # Adjust if grid dimensions are not equal
                verts, faces, _, _ = marching_cubes(density_grid, isovalue, spacing=spacing)
                
                # Create mesh and plot
                if len(faces) > 0:  # Only create mesh if we found faces
                    mesh = Poly3DCollection(verts[faces])
                    mesh.set_edgecolor('none')
                    mesh.set_facecolor(color)
                    mesh.set_alpha(alpha)
                    ax.add_collection3d(mesh)
            except ValueError as e:
                print(f"  Warning: Could not generate isosurface at {isovalue:.3f}: {str(e)}")
                continue
            except RuntimeError as e:
                print(f"  Warning: Runtime error at isosurface {isovalue:.3f}: {str(e)}")
                continue
        
        # Set axis limits and labels
        ax.set_xlim(0, sites[0])
        ax.set_ylim(0, sites[1])
        ax.set_zlim(0, sites[2])
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        
        # Adjust view angle and title
        ax.view_init(elev=view_angles[0], azim=view_angles[1])
        ax.set_title(f'Timestep {timestep}')
    
    # Add colorbar in the third column
    cbar_ax = fig.add_subplot(gs[:, -1])
    norm = plt.Normalize(global_min, global_max)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    
    # Create nice round ticks for colorbar
    ticks = np.linspace(global_min, global_max, 6)  # 6 evenly spaced ticks
    plt.colorbar(sm, cax=cbar_ax, label='Density', 
                format='%.3f',  # 3 decimal places
                ticks=ticks)  # Use evenly spaced ticks
    
    # Adjust spacing between subplots
    plt.subplots_adjust(wspace=0.3, hspace=0.3)
    
    # Save figure
    plt.savefig(output_fig, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close()

In [6]:
# Run both simulations
_ = qlbmlib.simulate_flow_classical(initial_dist_3d, config_3d, classical_csv)
_ = qlbmlib.simulate_flow(initial_dist_3d, config_3d, quantum_csv, True)

Classical simulation: iterations 0-10/10
Classical Iteration 1/10
Classical Iteration 2/10
Classical Iteration 3/10
Classical Iteration 4/10
Classical Iteration 5/10
Classical Iteration 6/10
Classical Iteration 7/10
Classical Iteration 8/10
Classical Iteration 9/10
Classical Iteration 10/10
Classical simulation complete. Results saved to experiments/3DQ27Planes_20250607_154613/classical.csv
Circuit configuration: will run iterations 0-10/10 with this configuration
Iteration 1 running...
Iteration 1/10:
  Compilation: 5.044 seconds
  Execution: 2.512 seconds
  Total: 7.555 seconds
Iteration 2 running...
Iteration 1/10:
  Compilation: 5.044 seconds
  Execution: 2.512 seconds
  Total: 7.555 seconds
Iteration 2 running...
Iteration 2/10:
  Compilation: 5.596 seconds
  Execution: 2.602 seconds
  Total: 8.198 seconds
Iteration 3 running...
Iteration 2/10:
  Compilation: 5.596 seconds
  Execution: 2.602 seconds
  Total: 8.198 seconds
Iteration 3 running...
Iteration 3/10:
  Compilation: 6.205

In [19]:
save_isosurface_plots(classical_csv, sites_3d ,f"{dirname}/classical_snapshots.png", timesteps=[0, 3, 6, 9], n_surfaces=4)
save_isosurface_plots(quantum_csv, sites_3d, f"{dirname}/quantum_snapshots.png", timesteps=[0,3,6,9], n_surfaces=4)
qlbmlib.save_rmse_comparison(classical_csv, quantum_csv, sites_3d, rmse_plot)

Time step 0:
  Density range: [0.100, 0.900]
  Using isosurface levels: ['0.121', '0.213', '0.376', '0.662']
Time step 3:
  Density range: [0.069, 0.995]
  Using isosurface levels: ['0.121', '0.213', '0.376', '0.662']
Time step 6:
  Density range: [0.083, 1.144]
  Using isosurface levels: ['0.121', '0.213', '0.376', '0.662']
Time step 9:
  Density range: [0.100, 1.167]
  Using isosurface levels: ['0.121', '0.213', '0.376', '0.662']
Time step 0:
  Density range: [0.100, 0.900]
  Using isosurface levels: ['0.121', '0.213', '0.376', '0.662']
Time step 3:
  Density range: [0.069, 0.994]
  Using isosurface levels: ['0.121', '0.213', '0.376', '0.662']
Time step 6:
  Density range: [0.083, 1.144]
  Using isosurface levels: ['0.121', '0.213', '0.376', '0.662']
Time step 9:
  Density range: [0.100, 1.167]
  Using isosurface levels: ['0.121', '0.213', '0.376', '0.662']
Time step 0:
  Density range: [0.100, 0.900]
  Using isosurface levels: ['0.121', '0.213', '0.376', '0.662']
Time step 3:
  Dens